USING TWITTER TO COLLECT TWEETS BY BY UGANDAN "INFLUENCERS" TO DO SOME NATURAL LANGUAGE PROCESSING AND OR SENTIMENT ANALYSIS

In [3]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

import tweepy
import csv
import json

# load Twitter API credentials

with open('twitter_credentials.json') as cred_data:
    info = json.load(cred_data)
    api_key = info['API_KEY']
    api_secret = info['API_SECRET']
    access_token_key = info['ACCESS_TOKEN_KEY']
    access_token_secret = info['ACCESS_TOKEN_SECRET']

def get_all_tweets(screen_name):

    # Twitter allows access to only 3240 tweets via this method

    # Authorization and initialization

    auth = tweepy.OAuthHandler(api_key, api_secret)
    auth.set_access_token(access_token_key, access_token_secret)
    api = tweepy.API(auth)

    # initialization of a list to hold all Tweets

    all_the_tweets = []

    # We will get the tweets with multiple requests of 200 tweets each

    new_tweets = api.user_timeline(screen_name=screen_name, count=200)

    # saving the most recent tweets

    all_the_tweets.extend(new_tweets)

    # save id of 1 less than the oldest tweet

    oldest_tweet = all_the_tweets[-1].id - 1

    # grabbing tweets till none are left

    while len(new_tweets):
        # The max_id param will be used subsequently to prevent duplicates
        new_tweets = api.user_timeline(screen_name=screen_name,
        count=200, max_id=oldest_tweet)

        # save most recent tweets

        all_the_tweets.extend(new_tweets)

        # id is updated to oldest tweet - 1 to keep track

        oldest_tweet = all_the_tweets[-1].id - 1
        print ('...%s tweets have been downloaded so far' % len(all_the_tweets))

        # transforming the tweets into a 2D array that will be used to populate the csv

        outtweets = [[tweet.id_str, tweet.created_at,
        tweet.text.encode('utf-8')] for tweet in all_the_tweets]

        # writing to the csv file

        with open('tweets/'  + screen_name + '.csv', 'w', encoding='utf8') as f:
            writer = csv.writer(f)
            writer.writerow(['id', 'created_at', 'text'])
            writer.writerows(outtweets)

if __name__ == '__main__':

# Enter the twitter handle of the person concerned

    get_all_tweets(input("Enter the twitter handle of the person whose tweets you want to download:- "))

Enter the twitter handle of the person whose tweets you want to download:- AlumBaeOfficial
...400 tweets have been downloaded so far
...600 tweets have been downloaded so far
...800 tweets have been downloaded so far
...1000 tweets have been downloaded so far
...1200 tweets have been downloaded so far
...1399 tweets have been downloaded so far
...1599 tweets have been downloaded so far
...1799 tweets have been downloaded so far
...1999 tweets have been downloaded so far
...2197 tweets have been downloaded so far
...2393 tweets have been downloaded so far
...2585 tweets have been downloaded so far
...2784 tweets have been downloaded so far
...2983 tweets have been downloaded so far
...3178 tweets have been downloaded so far
...3226 tweets have been downloaded so far
...3226 tweets have been downloaded so far


In [ ]:
'''
1f AldrineEsther
1m PatoShanquels
2m Owishemwe
3m Genoeric
4m PirateMulwana
5m Sempijjastanle
2f Ugawoman
6m AndrewOmuge
7m EddiePages
8m Fineboykera
9m ki jesseology
10m howard jeff
11m travor ug
3f Cecilia97
4f lilbooty
5f ghostbae
6f methialydiaN
7f evlyn_nakayi
8f jonam's princess
9f praise atukunda

merge into one dataset and create a tweep column
also search for all twitter accounts by ugandans with more than 10k followers and a follower/following ratio of
more than 2
measure activity through frequency of tweets
measure how original tweeps are by looking and the number of tweets
assess how devoted followers are by comparing likes,retweets and comments to influencers' original tweets
'''

In [18]:
import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings 
import seaborn as sns
from matplotlib import style
import matplotlib.pyplot as plt
from nltk.stem.porter import *

style.use('fivethirtyeight')
warnings.filterwarnings("ignore", category=DeprecationWarning)

x = pd.read_csv('tweets/AlumBaeOfficial.csv')

In [6]:
x.head()

,id,created_at,text
0,1176772802420101121,2019-09-25 08:17:54,b'How I will be swimming inside Pearl of Afric...
1,1176770079834484736,2019-09-25 08:07:05,b'If you have never been to Pearl of Africa ho...
2,1176768842997125121,2019-09-25 08:02:10,"b""Do y'all believe that people's childhoods af..."
3,1176582501051109376,2019-09-24 19:41:42,b'@LZuhael We deya #ThursdayPoaNights \xf0\x9f...
4,1176572839098486792,2019-09-24 19:03:19,b'girls who shoot their shot first got the bes...


In [17]:
#  below is a user-defined function to remove unwanted text patterns from the tweets
def remove_pattern(input_txt, pattern):
    '''It takes two arguments, one is the original string of text and the other is the 
        pattern of text that we want to remove from the string.'''
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt    

# create a new column tidy_tweet, it will contain the cleaned and processed tweets
# remove twitter handles (@user)
x['tidy_tweet'] = np.vectorize(remove_pattern)(x['text'], "@[\w]*")

# Removing Punctuations, Numbers, and Special Characters
x['tidy_tweet'] = x['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")

# Removing Short Words
x['tidy_tweet'] = x['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

# Tokenization
tokenized_tweet = x['tidy_tweet'].apply(lambda x: x.split())

# Stemming
stemmer = PorterStemmer()

tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming
tokenized_tweet.head()

0    [will, swimming, inside, Pearl, Africa, Hotel,...
1    [have, never, been, Pearl, Africa, hotel, like...
2    [believe, that, people, childhoods, affect, th...
3                           [deya, #ThursdayPoaNights]
4      [girls, shoot, their, shot, first, best, pussy]
Name: tidy_tweet, dtype: object

In [16]:
x.head()

,id,created_at,text,tidy_tweet
0,1176772802420101121,2019-09-25 08:17:54,b'How I will be swimming inside Pearl of Afric...,will swimming inside Pearl Africa Hotel come t...
1,1176770079834484736,2019-09-25 08:07:05,b'If you have never been to Pearl of Africa ho...,have never been Pearl Africa hotel like pull t...
2,1176768842997125121,2019-09-25 08:02:10,"b""Do y'all believe that people's childhoods af...",believe that people childhoods affect their re...
3,1176582501051109376,2019-09-24 19:41:42,b'@LZuhael We deya #ThursdayPoaNights \xf0\x9f...,deya #ThursdayPoaNights
4,1176572839098486792,2019-09-24 19:03:19,b'girls who shoot their shot first got the bes...,girls shoot their shot first best pussy
